### Import libraries and data

In [0]:
import pandas as pd
import csv
import re
import numpy as np

In [0]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [0]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [0]:
unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Calculations

In [0]:
from sklearn import linear_model, svm
regVols = linear_model.LinearRegression()
regAlpha = linear_model.LinearRegression()

In [0]:
alpha = 0.00
x2=pd.DataFrame()
for step in range(1,50):
    print step
    regVols.fit(unitformula, vols.div(temps.Temperature*alpha+1, axis="index"))
    w = regVols.coef_
    for line in zip(unitformula.columns.values, w[0]):
        x2[line[0]] = unitformula[line[0]].apply(lambda x: x*line[1])
    x2["sum"] = x2.sum(axis=1)
    vol2 = vols.div(x2["sum"], axis="index") - 1
    regAlpha.fit(temps, vol2)
    alpha = regAlpha.coef_[0][0]
    #unitformula["Keep"] = x2.apply(percCalc, args=(vols), axis=1)
    #print "Element average volume (cubic angstroms)", zip(unitformula.columns.values,w[0])
    print "Alpha Value: ", alpha

In [0]:
def percCalc (row):
    predictedvolume = row["sum"]
    vols = row["Volume"]
    percentage = predictedvolume / vols
    if (percentage >=0.95) and (percentage <=1.05):
        return 1
    else:
        return 0

unitformula["Keep"] = pd.concat([x2,vols], axis=1).apply(percCalc, axis=1)
print unitformula["Keep"].sum()

Here are attempts at improving the percentage function; now the predictedvolume and actual volumes are defined by the row they are contained in , in the dataframe and the only item passed are the rows.

At the bottom we can see that the predicted volumes and actual volumes are concatinated (joined) by an axis, and to this resultant dataframe the percentage function is applied; the results are passed to a new column in the unitformula dataframe called "KEEP".

This piece of code is here rather than before the loop because this was just a test to get it working, rather than a finished version.
